In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing the library 
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV



In [ ]:
%%time
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use

In [ ]:
#For generating 0 or 1 on the basis of resp features(which are returns from the market)
train['action'] =  ( ((train['resp_1'] > 0 ) | (train['resp_2'] > 0 ) | (train['resp_3'] > 0 ) | (train['resp_4'] > 0 )) & (train['resp'] > 0 ) ).astype('int')

In [ ]:
x_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

# ***1 stage of hyper-param tuning: tuning model complexity***

In [ ]:
param_grid = {'num_leaves': [7, 15, 31, 80,120], 
              'max_depth': [5, 6, -1,7,8],
             'n_estimators': [100, 200, 300,500,600]}

In [ ]:
clf = lgb.LGBMClassifier
grid_searcher = GridSearchCV(estimator=clf, param_grid=param_grid, 
                             cv=5, verbose=1, n_jobs=4)


In [ ]:

#Hypertuned Model
clf = lgb.LGBMClassifier(boosting_type='gbdt',
                         num_leaves=50, 
                         max_depth=- 1,
                         learning_rate=0.07,
                         n_estimators=1000, 
                         subsample_for_bin=200000,
                         objective='binary',
                         metric='auc', 
                         min_split_gain=0.5,
                         min_child_weight=1,
                         min_child_samples=5, 
                         subsample=0.7,
                         subsample_freq=1,
                         colsample_bytree=0.75, 
                         reg_alpha=1.2,
                         reg_lambda=1.2,
                         random_state=42,
                         n_jobs=- 1, 
                         silent=True,
                         importance_type='split')

Original 
#Hypertuned Model
clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=124, 
                         max_depth=- 1, learning_rate=0.07, n_estimators=50, 
                         subsample_for_bin=200000, objective='binary', metric='auc', 
                         min_split_gain=0.5, min_child_weight=1, min_child_samples=5, 
                         subsample=0.7, subsample_freq=1, colsample_bytree=0.75, 
                         reg_alpha=1.2, reg_lambda=1.2, random_state=42, n_jobs=- 1, 
                         silent=True, importance_type='split')

In [ ]:
%time clf.fit(x_train, y_train)

In [ ]:
from tqdm import tqdm
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)